<a href="https://colab.research.google.com/github/ArthurVarteressians/IRIS/blob/master/IRIS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.metrics import roc_auc_score

from torch import nn
import torch.optim as optim
import torch
from torch.utils.data import TensorDataset, DataLoader, Dataset

In [ ]:
data = pd.read_csv("https://gist.githubusercontent.com/curran/a08a1080b88344b0c8a7/raw/639388c2cbc2120a14dcf466e85730eb8be498bb/iris.csv")

In [ ]:
data.shape

(150, 5)

In [ ]:
data.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data.drop(["species"], axis = 1).values, data["species"].values, test_size=0.20, random_state=42)

In [ ]:
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
X_train = torch.from_numpy(X_train)
X_test = torch.from_numpy(X_test)

In [ ]:
enc = OneHotEncoder(handle_unknown='ignore')
enc.fit(y_train.reshape(-1,1))
y_train = enc.transform(y_train.reshape(-1, 1)).toarray()
y_train = torch.from_numpy(y_train)

y_test = enc.transform(y_test.reshape(-1, 1)).toarray()
y_test = torch.from_numpy(y_test)

In [ ]:
dataset_train = TensorDataset(X_train, y_train)
train_dataloader = DataLoader(dataset_train, batch_size=24)

In [ ]:
dataset_test = TensorDataset(X_test, y_test)
test_dataloader = DataLoader(dataset_test, batch_size=24)

In [ ]:
model = nn.Sequential(
        nn.Linear(4,4),
        nn.ReLU(),
        nn.Linear(4,3),
        nn.LogSoftmax()
        )

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [ ]:
num_epoch = 200
for epoch in range(num_epoch):  # loop over the dataset multiple times

    train_loss = 0.0
    test_loss = 0.0

    for (x0, y0) in train_dataloader:
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs_train = model(x0.float())
        loss_train = criterion(outputs_train, torch.max(y0, 1)[1])
        loss_train.backward()
        optimizer.step()

        # print statistics
        train_loss += loss_train.item()


    with torch.no_grad():
        for (x1, y1) in test_dataloader:
            outputs_test = model(x1.float())
            loss_test = criterion(outputs_test, torch.max(y1, 1)[1])
            test_loss += loss_test.item()


    print([train_loss, test_loss])

print('Finished Training')

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:117: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


[5.6259084939956665, 2.3010056018829346]
[5.6063233613967896, 2.2870051860809326]
[5.574748516082764, 2.268592596054077]
[5.53633189201355, 2.247750997543335]
[5.494060397148132, 2.2256311178207397]
[5.449615240097046, 2.2028759717941284]
[5.403919696807861, 2.179830312728882]
[5.3574525117874146, 2.156665325164795]
[5.310516953468323, 2.1334619522094727]
[5.263281941413879, 2.110242009162903]
[5.215726256370544, 2.0869970321655273]
[5.167974472045898, 2.0636775493621826]
[5.119899272918701, 2.040290117263794]
[5.071674406528473, 2.0168110132217407]
[5.023104012012482, 1.9932024478912354]
[4.974039256572723, 1.9695901274681091]
[4.924418687820435, 1.9458413124084473]
[4.874313116073608, 1.921953558921814]
[4.823713302612305, 1.8978821635246277]
[4.772643983364105, 1.8736572265625]
[4.7210357785224915, 1.849264919757843]
[4.668895781040192, 1.8247198462486267]
[4.61640739440918, 1.8000611066818237]
[4.5635271072387695, 1.775343418121338]
[4.510286867618561, 1.7505286931991577]
[4.456780

In [ ]:
pred_train = torch.exp(model(X_train.float()))
pred_test = torch.exp(model(X_test.float()))

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:117: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


In [ ]:
roc_auc_score(y_train.numpy(), pred_train.detach().numpy(), average = None)

array([1.        , 0.97036122, 0.97119342])

In [ ]:
roc_auc_score(y_test.numpy(), pred_test.detach().numpy(), average = None)

array([1., 1., 1.])